In [1]:
import numpy
from time import time
from matplotlib import pyplot
from codes.prism import prism_tf
from codes.grids import regular_grid
from codes.plot import draw_prism
from scipy.interpolate import griddata
from codes.auxiliars import noise_normal_dist
from codes.auxiliars import rotate3D_xyz
from codes.statistical import analysis
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
from codes.statistical import cccoef
from codes.filtering import reduction
from codes.derivative import totalgrad, zderiv
from codes.equivalentlayer import layer, fit_layer, rtp_layer

In [2]:
nx, ny = 40, 40
area = (-5000., 5000., -5000., 5000.)
shape = (nx, ny)
xo, yo, zo = regular_grid(area, shape, -100.)

# Teste de validação

In [3]:
intensity = 2.
inc, dec, incs, decs = 0., 0., 0., 0. 

In [ ]:
# Define modelo e direcoes
source1 = [0., 500., -2000., 2000., 10., 250.]
source2 = [-500., 0., -2000., 2000., 200., 500.]
source3 = [-1000., -500., -2000., 2000., 450., 750.]
source4 = [-1500., -1000., -2000., 2000., 700., 1000.]
source5 = [-2000., -1500., -2000., 2000., 900., 1250.]
source6 = [-2500., -2000., -2000., 2000., 1200., 1500.]

In [ ]:
figure = pyplot.figure(figsize=(8,8))
ax = figure.gca(projection = '3d')

x1a, x2a, y1a, y2a, z1a, z2a = source1
x1b, x2b, y1b, y2b, z1b, z2b = source2
x1c, x2c, y1c, y2c, z1c, z2c = source3
x1d, x2d, y1d, y2d, z1d, z2d = source4
x1e, x2e, y1e, y2e, z1e, z2e = source5
x1f, x2f, y1f, y2f, z1f, z2f = source6
v1a = numpy.array([[x1a, y1a, z2a], [x1a, y2a, z2a], [x2a, y2a, z2a], [x2a, y1a, z2a], 
                 [x1a, y1a, z1a], [x1a, y2a, z1a], [x2a, y2a, z1a], [x2a, y1a, z1a]])
v2b = numpy.array([[x1b, y1b, z2b], [x1b, y2b, z2b], [x2b, y2b, z2b], [x2b, y1b, z2b], 
                 [x1b, y1b, z1b], [x1b, y2b, z1b], [x2b, y2b, z1b], [x2b, y1b, z1b]])
v3a = numpy.array([[x1c, y1c, z2c], [x1c, y2c, z2c], [x2c, y2c, z2c], [x2c, y1c, z2c], 
                 [x1c, y1c, z1c], [x1c, y2c, z1c], [x2c, y2c, z1c], [x2c, y1c, z1c]])
v4d = numpy.array([[x1d, y1d, z2d], [x1d, y2d, z2d], [x2d, y2d, z2d], [x2d, y1d, z2d], 
                 [x1d, y1d, z1d], [x1d, y2d, z1d], [x2d, y2d, z1d], [x2d, y1d, z1d]])
v5e = numpy.array([[x1e, y1e, z2e], [x1e, y2e, z2e], [x2e, y2e, z2e], [x2e, y1e, z2e], 
                 [x1e, y1e, z1e], [x1e, y2e, z1e], [x2e, y2e, z1e], [x2e, y1e, z1e]])
v6f = numpy.array([[x1f, y1f, z2f], [x1f, y2f, z2f], [x2f, y2f, z2f], [x2f, y1f, z2f], 
                 [x1f, y1f, z1f], [x1f, y2f, z1f], [x2f, y2f, z1f], [x2f, y1f, z1f]])
vertices1 = [[v1a[0],v1a[1],v1a[2],v1a[3]], [v1a[0],v1a[1],v1a[5],v1a[4]], [v1a[1],v1a[2],v1a[6],v1a[5]],
            [v1a[2],v1a[3],v1a[7],v1a[6]], [v1a[3],v1a[0],v1a[4],v1a[7]], [v1a[4],v1a[5],v1a[6],v1a[7]]]
vertices2 = [[v2b[0],v2b[1],v2b[2],v2b[3]], [v2b[0],v2b[1],v2b[5],v2b[4]], [v2b[1],v2b[2],v2b[6],v2b[5]],
            [v2b[2],v2b[3],v2b[7],v2b[6]], [v2b[3],v2b[0],v2b[4],v2b[7]], [v2b[4],v2b[5],v2b[6],v2b[7]]]
vertices3 = [[v3a[0],v3a[1],v3a[2],v3a[3]], [v3a[0],v3a[1],v3a[5],v3a[4]], [v3a[1],v3a[2],v3a[6],v3a[5]],
            [v3a[2],v3a[3],v3a[7],v3a[6]], [v3a[3],v3a[0],v3a[4],v3a[7]], [v3a[4],v3a[5],v3a[6],v3a[7]]]
vertices4 = [[v4d[0],v4d[1],v4d[2],v4d[3]], [v4d[0],v4d[1],v4d[5],v4d[4]], [v4d[1],v4d[2],v4d[6],v4d[5]],
            [v4d[2],v4d[3],v4d[7],v4d[6]], [v4d[3],v4d[0],v4d[4],v4d[7]], [v4d[4],v4d[5],v4d[6],v4d[7]]]
vertices5 = [[v5e[0],v5e[1],v5e[2],v5e[3]], [v5e[0],v5e[1],v5e[5],v5e[4]], [v5e[1],v5e[2],v5e[6],v5e[5]],
            [v5e[2],v5e[3],v5e[7],v5e[6]], [v5e[3],v5e[0],v5e[4],v5e[7]], [v5e[4],v5e[5],v5e[6],v5e[7]]]
vertices6 = [[v6f[0],v6f[1],v6f[2],v6f[3]], [v6f[0],v6f[1],v6f[5],v6f[4]], [v6f[1],v6f[2],v6f[6],v6f[5]],
            [v6f[2],v6f[3],v6f[7],v6f[6]], [v6f[3],v6f[0],v6f[4],v6f[7]], [v6f[4],v6f[5],v6f[6],v6f[7]]]
pc5a = Poly3DCollection(vertices1, alpha = 1., linewidths = 0., edgecolors = 'k')
pc5a.set_facecolor('sienna')
ax.add_collection3d(pc5a)
pc5b = Poly3DCollection(vertices2, alpha = 1., linewidths = 0., edgecolors = 'k')
pc5b.set_facecolor('sienna')
ax.add_collection3d(pc5b)
pc5c = Poly3DCollection(vertices3, alpha = 1., linewidths = 0., edgecolors = 'k')
pc5c.set_facecolor('sienna')
ax.add_collection3d(pc5c)
pc5d = Poly3DCollection(vertices4, alpha = 1., linewidths = 0., edgecolors = 'k')
pc5d.set_facecolor('sienna')
ax.add_collection3d(pc5d)
pc5e = Poly3DCollection(vertices5, alpha = 1., linewidths = 0., edgecolors = 'k')
pc5e.set_facecolor('sienna')
ax.add_collection3d(pc5e)
pc5f = Poly3DCollection(vertices6, alpha = 1., linewidths = 0., edgecolors = 'k')
pc5f.set_facecolor('sienna')
ax.add_collection3d(pc5f)

x_scale = 1.
y_scale = 1.
z_scale = 1.
scale=numpy.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3] = 1.
def short_proj():
    return numpy.dot(Axes3D.get_proj(ax), scale)
ax.get_proj=short_proj
ax.set_xlabel('North coordinate (m)', size = 12, labelpad = 20)
ax.set_ylabel('East coordinate (m)', size = 12, labelpad = 20)
ax.set_zlabel('Depth(m)', size = 12, labelpad = 15)
ax.set_xlim(xo.min(), xo.max())
ax.set_ylim(yo.min(), yo.max())
ax.set_zlim(0., 1500.)
ax.set_xticks(numpy.arange(xo.min(), xo.max(), 2500))
ax.set_yticks(numpy.linspace(yo.min(), yo.max(), 5))
ax.set_zticks(numpy.linspace(0., 1500., 6))
ax.view_init(210, 235)
pyplot.tight_layout(True)
pyplot.savefig('cap04b-figura01.png', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.savefig('cap04b-figura01.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.show()

In [ ]:
xr, yr, zr = rotate3D_xyz(xo, yo, zo, 0.)

In [ ]:
tf = prism_tf(xr, yr, zo, source1, intensity, inc, dec, incs, decs) + \
        prism_tf(xr, yr, zo, source2, intensity, inc, dec, incs, decs) + \
        prism_tf(xr, yr, zo, source3, intensity, inc, dec, incs, decs) + \
        prism_tf(xr, yr, zo, source4, intensity, inc, dec, incs, decs) + \
        prism_tf(xr, yr, zo, source5, intensity, inc, dec, incs, decs) + \
        prism_tf(xr, yr, zo, source6, intensity, inc, dec, incs, decs)

In [ ]:
print 'Anomalia magnetica:'
_ = analysis(tf, 'nT')

In [ ]:
tf = noise_normal_dist(tf, 5., 0.1)

In [ ]:
pyplot.figure(figsize=(8,6))
cs = pyplot.contourf(yo.reshape(shape), xo.reshape(shape), tf.reshape(shape), 
                50, cmap = pyplot.cm.RdBu)
#draw_prism(dike[:4], linewidth = 0.25, style = 'k--', xy2ne = True)
pyplot.text(-4800., -4700., '$I_o, D_o = [0^o, 0^o]$', 
            fontsize = 14.,bbox=dict(facecolor='white', edgecolor='k', alpha = 0.5))
pyplot.xlabel('East coordinate (m)', fontsize = 14, labelpad = 10)
pyplot.ylabel('North coordinate (m)', fontsize = 14, labelpad = 10)
pyplot.xticks(numpy.linspace(yo.min(), yo.max(), 5), fontsize = 12)
pyplot.yticks(numpy.linspace(xo.min(), xo.max(), 5), fontsize = 12)
cb = pyplot.colorbar(cs, aspect = 30, fraction = 0.04, orientation = 'vertical', pad = 0.05)
cb.set_ticks(numpy.linspace(tf.min(), tf.max(), 10))
cb.set_clim(tf.min(), tf.max())
cb.set_label('nT', fontsize = 14, labelpad = 5)
pyplot.tight_layout(True)

pyplot.savefig('cap04b-figura02.png', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.savefig('cap04b-figura02.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.show()

In [ ]:
area_layer = area
shape_layer = (15, 15)
level = 1200.

In [ ]:
xlayer, ylayer, zlayer = regular_grid(area, shape_layer, level)
eqlayer = layer(area_layer, shape_layer, level)
data = [xo, yo, zo, tf]
_, tf_pred = fit_layer(data, shape, eqlayer, shape_layer, 0.1, inc, dec, incs, decs)

In [ ]:
figure = pyplot.figure(figsize=(8,7))
ax = figure.gca(projection = '3d')
p1 = ax.contourf(xo.reshape(shape), yo.reshape(shape), tf.reshape(shape),
                 50, zdir='z', offset = 0., cmap = 'RdBu',
                  vmin = numpy.min(tf), vmax = numpy.max(tf))
cbar1 = pyplot.colorbar(p1, aspect = 25, fraction = 0.02, orientation = 'vertical', pad = -0.05)
cbar1.set_label('Observed data (nT)', fontsize = 14, rotation = 90.)
cbar1.set_ticks(numpy.linspace(tf.min(), tf.max(), 6))

p2 = ax.contourf(xo.reshape(shape), yo.reshape(shape), tf_pred.reshape(shape),
                 50, zdir='z', offset = level, cmap = 'RdBu',
                  vmin = numpy.min(tf_pred), vmax = numpy.max(tf_pred))
ax.scatter3D(xlayer.reshape(shape_layer), ylayer.reshape(shape_layer), zlayer.reshape(shape_layer), 
             s = 0.5, c = 'k')
cbar2 = pyplot.colorbar(p2, aspect = 30, fraction = 0.025, orientation = 'horizontal', pad = 0.05)
cbar2.set_label('Predicted data (nT)', fontsize = 14, rotation = 0.)
cbar2.set_ticks(numpy.linspace(tf_pred.min(), tf_pred.max(), 6))

x_scale = 1.
y_scale = 1.
z_scale = 1.25
scale=numpy.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3] = 1.
def short_proj():
    return numpy.dot(Axes3D.get_proj(ax), scale)
ax.get_proj=short_proj
ax.set_xlabel('North coordinate (m)', size = 12, labelpad = 20)
ax.set_ylabel('East coordinate (m)', size = 12, labelpad = 20)
ax.set_zlabel('Depth (m)', size = 12, labelpad = 20)
ax.set_xlim(xo.min(), xo.max())
ax.set_xticks(numpy.arange(xo.min(), xo.max(), 2000.))
ax.set_yticks(numpy.linspace(yo.min(), yo.max(), 5))
ax.set_zticks(numpy.linspace(0., level, 5))
ax.set_ylim(yo.min(), yo.max())
ax.set_zlim(0., level)
ax.view_init(210, 135)
pyplot.tight_layout(True)

pyplot.savefig('cap04b-figura03.png', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.savefig('cap04b-figura03.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.show()

In [ ]:
residuo = tf - tf_pred

In [ ]:
figure = pyplot.figure(figsize=(8,6))
ax = figure.gca(projection = '3d')
#ax.set_title('Ajuste de dados via camada equivalente', size = 16, pad = 10)
p1 = ax.contourf(xo.reshape(shape), yo.reshape(shape), residuo.reshape(shape),
                 50, zdir='z', offset = 0., cmap = 'RdBu',
                  vmin = numpy.min(residuo), vmax = numpy.max(residuo))
cbar1 = pyplot.colorbar(p1, aspect = 25, fraction = 0.02, orientation = 'vertical', pad = -0.05)
cbar1.set_label('$D_{obs} - D_{pred}$ (nT)', fontsize = 14, rotation = 90.)
cbar1.set_ticks(numpy.linspace(residuo.min(), residuo.max(), 6))

p2 = ax.contourf(xo.reshape(shape), yo.reshape(shape), tf_pred.reshape(shape),
                 50, zdir='z', offset = level, cmap = 'RdBu',
                  vmin = numpy.min(tf_pred), vmax = numpy.max(tf_pred))
ax.scatter3D(xlayer.reshape(shape_layer), ylayer.reshape(shape_layer), zlayer.reshape(shape_layer), 
             s = 1.5, c = 'k')
cbar2 = pyplot.colorbar(p2, aspect = 30, fraction = 0.025, orientation = 'horizontal', pad = 0.05)
cbar2.set_label('Predicted data (nT)', fontsize = 14, rotation = 0.)
cbar2.set_ticks(numpy.linspace(tf_pred.min(), tf_pred.max(), 6))

x_scale = 1.
y_scale = 1.
z_scale = 1.25
scale=numpy.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3] = 1.
def short_proj():
    return numpy.dot(Axes3D.get_proj(ax), scale)
ax.get_proj=short_proj
ax.set_xlabel('North coordinate (m)', size = 12, labelpad = 20)
ax.set_ylabel('East coordinate (m)', size = 12, labelpad = 20)
ax.set_zlabel('Depth (m)', size = 12, labelpad = 20)
ax.set_xlim(xo.min(), xo.max())
ax.set_xticks(numpy.arange(xo.min(), xo.max(), 2000.))
ax.set_yticks(numpy.linspace(yo.min(), yo.max(), 5))
ax.set_zticks(numpy.linspace(0., level, 5))
ax.set_ylim(yo.min(), yo.max())
ax.set_zlim(0., level)
ax.view_init(210, 135)
pyplot.tight_layout(True)

pyplot.savefig('cap04b-figura04.png', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.savefig('cap04b-figura04.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.show()

In [ ]:
xobs = xo.reshape(shape)
yobs = yo.reshape(shape)
zobs = zo.reshape(shape)
dobs = tf.reshape(shape)

In [ ]:
incfull = numpy.linspace(-90., 90., 37)
decfull = numpy.linspace(-90., 90., 13)
Dfull, Ifull = numpy.meshgrid(decfull, incfull)
resfull = numpy.zeros_like(Dfull)

In [ ]:
initial = time()
for k2, d in enumerate(decfull):
    for k1, i in enumerate(incfull):
        if i >= -15. or i <= 15.:
            rtp = rtp_layer(data, shape, eqlayer, shape_layer, 0.01, inc, dec, i, d)
            resfull[k1,k2] = cccoef(totalgrad(xobs, yobs, rtp.reshape(shape)),
                                     zderiv(xobs, yobs, rtp.reshape(shape)))    
        else:
            rtp = reduction(xobs, yobs, dobs, inc, dec, i, d)
            resfull[k1,k2] = cccoef(totalgrad(xobs, yobs, rtp),
                                     zderiv(xobs, yobs, rtp))            
final = time()
print 'Tempo de calculo (sec): %.3f' % (final-initial)

In [ ]:
print 'Maxima correlacao:', resfull.max()
print 'Minima correlacao:', resfull.min()

In [ ]:
imaxfull, jmaxfull = numpy.where(resfull == resfull.max())
incmaxfull = float(Ifull[imaxfull, jmaxfull])
decmaxfull = float(Dfull[imaxfull, jmaxfull])
print '(I,D) maximo:', incmaxfull, decmaxfull

In [ ]:
pyplot.figure(figsize=(8, 7))
fig5c = pyplot.contour(Dfull, Ifull, resfull, 15, colors = 'k', linestyle = 'solid', linewidths = 1)
pyplot.contourf(Dfull, Ifull, resfull, 15, cmap = pyplot.cm.gray)
pyplot.clabel(fig5c, fontsize = 8)
pyplot.plot(decs, incs, 'r*', label = 'Source direction', markersize = 15)
pyplot.plot(decmaxfull, incmaxfull, 'bs', label = 'Calculated direction', markersize = 8)
pyplot.xlabel('Declination $(^o)$', fontsize = 14)
pyplot.ylabel('Inclination $(^o)$', fontsize = 14)
pyplot.xticks(numpy.linspace(Dfull.min(), Dfull.max(), 7), fontsize = 14)
pyplot.yticks(numpy.linspace(Ifull.min(), Ifull.max(), 7), fontsize = 14)
pyplot.legend(loc = 'upper left', fontsize = 'x-large')
pyplot.colorbar()
pyplot.grid(color = 'k', linestyle='--', linewidth = .5)
pyplot.savefig('cap04b-figura05.png', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.savefig('cap04b-figura05.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.show()

In [ ]:
areai = (-90., 90., -90., 90)
shapei = (181, 181)
Inci, Deci = regular_grid(areai, shapei)
datares = resfull.reshape(resfull.size)

In [ ]:
datai = griddata((Ifull.reshape(Ifull.size), Dfull.reshape(Dfull.size)), 
                 datares, (Inci, Deci), method = 'cubic')
datai = datai.reshape(shapei)

In [ ]:
imaxi, jmaxi = numpy.where(datai == datai.max())
inci = float(Inci.reshape(shapei)[imaxi, jmaxi])
deci = float(Deci.reshape(shapei)[imaxi, jmaxi])
print '(I,D) maximo:', inci, deci

In [ ]:
pyplot.figure(figsize=(8, 7))
fig5c = pyplot.contour(Deci.reshape(shapei), Inci.reshape(shapei), datai, 15, 
                       colors = 'k', linestyle = 'solid', linewidths = 1)
pyplot.contourf(Deci.reshape(shapei), Inci.reshape(shapei), datai, 15, cmap = pyplot.cm.gray)
pyplot.clabel(fig5c, fontsize = 8)
pyplot.plot(decs, incs, 'r*', label = 'Source direction', markersize = 15)
pyplot.plot(deci, inci, 'bs', label = 'Calculated direction', markersize = 8)
pyplot.xlabel('Declination $(^o)$', fontsize = 14)
pyplot.ylabel('Inclination $(^o)$', fontsize = 14)
pyplot.xticks(numpy.linspace(Deci.min(), Deci.max(), 7), fontsize = 14)
pyplot.yticks(numpy.linspace(Inci.min(), Inci.max(), 7), fontsize = 14)
pyplot.legend(loc = 'upper center', fontsize = 'large')
pyplot.colorbar()
pyplot.grid(color = 'k', linestyle='--', linewidth = .25)
pyplot.savefig('cap04b-figura06.png', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.savefig('cap04b-figura06.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.show()

In [ ]:
rtp_final = rtp_layer(data, shape, layer(area_layer, (40,40), 800.), (40,40), 
                      0.005, inc, dec, incmaxfull, decmaxfull)

In [ ]:
pyplot.figure(figsize=(8,6))
cs = pyplot.contourf(yo.reshape(shape), xo.reshape(shape), rtp_final.reshape(shape), 
                     20, cmap = pyplot.cm.RdBu)
pyplot.xlabel('East coordinate (m)', fontsize = 14, labelpad = 10)
pyplot.ylabel('North coordinate (m)', fontsize = 14, labelpad = 10)
pyplot.xticks(numpy.linspace(yo.min(), yo.max(), 5), fontsize = 12)
pyplot.yticks(numpy.linspace(xo.min(), xo.max(), 5), fontsize = 12)

cb = pyplot.colorbar(cs, aspect = 30, fraction = 0.04, orientation = 'vertical', pad = 0.05)
cb.set_ticks(numpy.linspace(rtp_final.min(), rtp_final.max(), 10))
cb.set_clim(rtp_final.min(), rtp_final.max())
cb.set_label('nT', fontsize = 14, labelpad = 0.)

pyplot.savefig('cap04b-figura07.png', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.savefig('cap04b-figura07.pdf', dpi = 300, bbox_inches = 'tight', transparent = True)
pyplot.show()